In [7]:
from datetime import date, datetime
from typing import Any, Optional

import httpx
import rich
from pydantic import BaseModel, Field, TypeAdapter
from tqdm import tqdm


class Member(BaseModel):
    MnisId: int
    PimsId: int
    Name: str
    ListAs: str
    Constituency: str
    Status: str
    Party: str
    PartyId: int
    PartyColour: Optional[str]
    PhotoUrl: str


class Sponsor(BaseModel):
    Id: int
    MemberId: int
    Member: Member
    SponsoringOrder: int
    CreatedWhen: datetime
    IsWithdrawn: bool
    WithdrawnDate: Optional[datetime]


class Motion(BaseModel):
    Sponsors: Optional[list[Sponsor]] = Field(default_factory=list)
    Amendments: list[str]
    Id: int
    Status: int
    StatusDate: datetime
    MemberId: int
    PrimarySponsor: Member
    Title: str
    MotionText: str
    AmendmentToMotionId: Optional[int]
    UIN: int
    AmendmentSuffix: Optional[str]
    DateTabled: datetime
    PrayingAgainstNegativeStatutoryInstrumentId: Optional[int] = None
    StatutoryInstrumentNumber: Optional[int] = None
    StatutoryInstrumentYear: Optional[int] = None
    StatutoryInstrumentTitle: Optional[str] = None
    UINWithAmendmentSuffix: str
    SponsorsCount: int


MotionList = TypeAdapter(list[Motion])


def paged_request(url: str, params: dict[str, Any]) -> list[dict]:
    """
    Fetches all pages of a paginated API request.
    This is an api with a take and a skip .
    The items we care about are in the "Response" key.
    There is an overall total in the PagingInfo.Total.
    """

    all_items = []
    skip = 0
    bar = None
    while True:
        full_params = params.copy()
        full_params["skip"] = skip
        full_params["take"] = 100
        response = httpx.get(url, params=full_params)
        response.raise_for_status()

        data = response.json()
        all_items.extend(data["Response"])

        skip += len(data["Response"])
        if not bar:
            bar = tqdm(total=data["PagingInfo"]["Total"])
        bar.update(len(data["Response"]))
        if skip >= data["PagingInfo"]["Total"]:
            break
    bar.close()
    return all_items


def get_edms(*, since: Optional[date] = None) -> list[Motion]:
    api_end_point = (
        "https://oralquestionsandmotions-api.parliament.uk/EarlyDayMotions/list"
    )

    params = {}
    if since:
        params["tabledStartDate"] = date.isoformat(since)
    data = paged_request(api_end_point, params)
    return MotionList.validate_python(data)


def get_single_edm(edm_id: int) -> Motion:
    api_end_point = (
        f"https://oralquestionsandmotions-api.parliament.uk/EarlyDayMotion/{edm_id}"
    )
    response = httpx.get(api_end_point)
    response.raise_for_status()
    data = response.json()
    return Motion.model_validate(data["Response"])


# data = get_edms(since=date(2019, 12, 12))
# data

rich.print(get_single_edm(62455))

Motion(
    Sponsors=[
        Sponsor(
            Id=2242739,
            MemberId=5277,
            Member=Member(
                MnisId=5277,
                PimsId=6693,
                Name='Graham Leadbitter',
                ListAs='Leadbitter, Graham',
                Constituency='Moray West, Nairn and Strathspey',
                Status='Active',
                Party='Scottish National Party',
                PartyId=29,
                PartyColour='fff685',
                PhotoUrl='https://members-api.parliament.uk/api/Members/5277/Portrait?cropTyp
e=OneOne&webVersion=true'
            ),
            SponsoringOrder=1,
            CreatedWhen=datetime.datetime(2024, 9, 3, 15, 16, 55, 86666),
            IsWithdrawn=False,
            WithdrawnDate=None
        ),
        Sponsor(
            Id=2242939,
            MemberId=178,
            Member=Member(
                MnisId=178,
                PimsId=1799,
                Name='John McDonnell',
                ListAs='McDonnell, John',
                Constituency='Hayes and Harlington',
                Status='Active',
                Party='Independent',
                PartyId=8,
                PartyColour='909090',
                PhotoUrl='https://members-api.parliament.uk/api/Members/178/Portrait?cropType
=OneOne&webVersion=true'
            ),
            SponsoringOrder=2,
            CreatedWhen=datetime.datetime(2024, 9, 6, 11, 23, 27, 183333),
            IsWithdrawn=False,
            WithdrawnDate=None
        )
    ],
    Amendments=[],
    Id=62455,
    Status=0,
    StatusDate=datetime.datetime(2024, 9, 5, 18, 8, 48, 107000),
    MemberId=5277,
    PrimarySponsor=Member(
        MnisId=5277,
        PimsId=6693,
        Name='Graham Leadbitter',
        ListAs='Leadbitter, Graham',
        Constituency='Moray West, Nairn and Strathspey',
        Status='Active',
        Party='Scottish National Party',
        PartyId=29,
        PartyColour='fff685',
        PhotoUrl='https://members-api.parliament.uk/api/Members/5277/Portrait?cropType=OneOne
&webVersion=true'
    ),
    Title='Armed Services Advice Project funding',
    MotionText='That the House notes with concern the reported decision by the Royal British 
Legion to cease funding to the Armed Services Advice Project, which will see the service 
close; understands that the Project was established in 2010 as a partnership between 
Poppyscotland and Citizens Advice Services Scotland to provide dedicated information, advice 
and support to members of the Armed Forces community in Scotland; believes that the Project 
supports an average of 2,200 beneficiaries a year, through 16 local Citizens Advice Bureaux 
across Scotland, covering 75% of all local authority areas in Scotland; notes that local CABs
are now getting referrals from Poppy Scotland, demonstrating the crucial need for the 
services; understands that, since its inception, the project has generated over £20 million 
of financial gain for veterans and their families; acknowledges recent analysis, which shows 
that 95% of service users showed improvements in peace of mind, 84% saw improvements in their
financial situation, including access to benefits, 75% felt more able to help themselves, and
67% saw improvements to their mental health, and notes the calls for the Legion, in light of 
what it sees as these significant outcomes, to reconsider the decision, which, it considers, 
will have a detrimental impact on the veteran community in Scotland.',
    AmendmentToMotionId=None,
    UIN=137,
    AmendmentSuffix=None,
    DateTabled=datetime.datetime(2024, 9, 5, 0, 0),
    PrayingAgainstNegativeStatutoryInstrumentId=None,
    StatutoryInstrumentNumber=None,
    StatutoryInstrumentYear=None,
    StatutoryInstrumentTitle=None,
    UINWithAmendmentSuffix='137',
    SponsorsCount=0
)